# BERT Model for Spam Review Classification

In this notebook, we fine-tune a **BERT-based model** on our Google reviews dataset to detect spam reviews.  

Unlike Isolation Forest and Autoencoder (which work on numeric/engineered features),  
BERT operates directly on the **raw review text**.  

- **Why BERT?**  
  BERT (Bidirectional Encoder Representations from Transformers) captures the **contextual meaning of words** in a sentence by looking both left and right.  
  This makes it highly effective for text classification tasks.  

Here, we use **DistilBERT**, a lighter and faster version of BERT, fine-tuned with a classification head for binary classification:  
- **0 = normal review**  
- **1 = spam review**


## Step 1: Importing Libraries

We import the essential packages:  
- **Transformers (Hugging Face):** for pretrained DistilBERT and tokenization  
- **Torch (PyTorch):** for training and evaluation  
- **Scikit-learn:** for metrics such as Accuracy, F1, ROC-AUC  
- **Pandas & Numpy:** for dataset handling  

These provide the core tools for NLP preprocessing, model building, and evaluation.


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from dataclasses import dataclass
from typing import Dict, Any
from inspect import signature

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, f1_score, roc_auc_score, average_precision_score, confusion_matrix
)

from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, DataCollatorWithPadding
)

In [ ]:
DATA_PATH   = "final_dataset_with_fusion.parquet"
TEXT_COL    = "text"
LABEL_COL   = "is_anomaly"
MODEL_NAME  = "distilbert-base-uncased"    
MAX_LEN     = 128
SEED        = 42
TEST_SIZE   = 0.20
VAL_SIZE    = 0.10                     # of the remaining after test split
BATCH_TRAIN = 8
BATCH_EVAL  = 16
LR          = 2e-5
EPOCHS      = 3

# If you're on Apple Silicon / MPS, avoid pin_memory warnings:
os.environ.setdefault("PYTORCH_ENABLE_MPS_FALLBACK", "1")

np.random.seed(SEED)
torch.manual_seed(SEED)

## Step 2: Splitting the Dataset

We split the dataset into **train, validation, and test sets**:  
- **Training set:** used to fine-tune the model weights  
- **Validation set:** monitors generalisation and prevents overfitting  
- **Test set:** final unbiased evaluation of model performance  

This ensures we fairly assess the model’s ability to generalise to unseen reviews.


In [ ]:
df = pd.read_parquet(DATA_PATH)
if not {TEXT_COL, LABEL_COL}.issubset(df.columns):
    raise KeyError(f"Input must contain columns: {TEXT_COL}, {LABEL_COL}")

df = df[[TEXT_COL, LABEL_COL]].dropna()
df[LABEL_COL] = df[LABEL_COL].astype(int)

train_df, test_df = train_test_split(
    df, test_size=TEST_SIZE, stratify=df[LABEL_COL], random_state=SEED
)
train_df, val_df = train_test_split(
    train_df, test_size=VAL_SIZE, stratify=train_df[LABEL_COL], random_state=SEED
)

print(f"[sizes] train={len(train_df)}  val={len(val_df)}  test={len(test_df)}")
print(f"[label balance] positives overall={df[LABEL_COL].mean():.3f}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.texts = list(texts)
        self.labels = list(labels)
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            truncation=True, padding=False, max_length=MAX_LEN
        )
        enc["labels"] = int(self.labels[idx])
        return {k: torch.tensor(v) for k, v in enc.items()}

train_ds = TextDataset(train_df[TEXT_COL], train_df[LABEL_COL])
val_ds   = TextDataset(val_df[TEXT_COL],   val_df[LABEL_COL])
test_ds  = TextDataset(test_df[TEXT_COL],  test_df[LABEL_COL])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Step 3: Loading and Configuring the Model

We load a **pretrained DistilBERT** model from Hugging Face.  

- The model comes with pretrained language representations (trained on billions of words).  
- We add a **classification head** on top with 2 outputs:  
  - **0 = normal review**  
  - **1 = spam review**  

Fine-tuning allows the model to adapt its language understanding to our spam detection task.


In [ ]:
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=num_labels
)

In [ ]:
def compute_metrics(p) -> Dict[str, Any]:
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    probs = torch.softmax(torch.tensor(preds), dim=1).numpy()
    y_pred = probs.argmax(axis=1)
    y_true = p.label_ids

    try:
        roc = roc_auc_score(y_true, probs[:, 1])
    except ValueError:
        roc = float("nan")
    try:
        ap = average_precision_score(y_true, probs[:, 1])
    except ValueError:
        ap = float("nan")

    return {
        "accuracy": float((y_pred == y_true).mean()),
        "f1": f1_score(y_true, y_pred, average="binary"),
        "roc_auc": roc,
        "pr_auc": ap,
    }

In [ ]:
def build_training_args(**prefs):
    sig = signature(TrainingArguments.__init__).parameters
    if "evaluation_strategy" in prefs and "evaluation_strategy" not in sig and "eval_strategy" in sig:
        prefs["eval_strategy"] = prefs.pop("evaluation_strategy")
    allowed = set(sig.keys())
    used = {k: v for k, v in prefs.items() if k in allowed}
    dropped = sorted(set(prefs) - set(used))
    if dropped:
        print("[warn] Dropped unsupported TrainingArguments keys:", dropped)
    return TrainingArguments(**used)

## Step 4: Defining Evaluation Metrics

We define a function to compute standard classification metrics:  

- **Accuracy:** overall proportion of correct predictions  
- **F1 Score:** balances precision and recall, important for imbalanced data  
- **ROC-AUC:** measures ability to separate normal vs spam across thresholds  
- **PR-AUC:** precision-recall tradeoff, useful when spam reviews are rare  

The function also handles edge cases where certain metrics are undefined (e.g., if a batch only contains one class).


In [ ]:
args = build_training_args(
    output_dir="./bert_text_model",
    evaluation_strategy="epoch",        # will auto-map to eval_strategy in 4.56
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,

    per_device_train_batch_size=BATCH_TRAIN,
    per_device_eval_batch_size=BATCH_EVAL,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    logging_steps=50,
    report_to="none",
    seed=SEED,
    dataloader_pin_memory=False,
    dataloader_num_workers=0,           # safer across OSes/Jupyter
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

In [ ]:
print("\n[eval] validation metrics:")
val_metrics = trainer.evaluate(val_ds)
print(val_metrics)

## Step 5: Model Evaluation

After training, we evaluate the fine-tuned model on the **test dataset**.  

- Predictions are compared with ground-truth labels  
- Metrics (Accuracy, F1, ROC-AUC, PR-AUC) are reported  
- Confusion matrices and probability distributions can also be visualised  

This step confirms how well BERT performs compared to Isolation Forest, Autoencoder, and Fusion approaches.


In [ ]:


print("\n[eval] test metrics & reports:")
raw_preds = trainer.predict(test_ds)
test_probs = torch.softmax(torch.tensor(raw_preds.predictions), dim=1).numpy()
test_pred  = test_probs.argmax(axis=1)
y_true     = raw_preds.label_ids

print(classification_report(y_true, test_pred, digits=4))
try:
    print("ROC AUC:", roc_auc_score(y_true, test_probs[:, 1]))
    print("PR  AUC:", average_precision_score(y_true, test_probs[:, 1]))
except ValueError:
    print("ROC/PR AUC: not defined (single class present)")

print("Confusion matrix:\n", confusion_matrix(y_true, test_pred))

In [ ]:
os.makedirs("./bert_text_model", exist_ok=True)
pd.DataFrame({
    "y_true": y_true,
    "y_pred": test_pred,
    "p_pos": test_probs[:, 1]
}).to_csv("./bert_text_model/test_predictions.csv", index=False)
print("\nSaved predictions to ./bert_text_model/test_predictions.csv")


/Users/minruilee/Desktop/tiktok hackathon/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[sizes] train=19167  val=2130  test=5325
[label balance] positives overall=0.050


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/tv/zkyj084138d4l2c6vf250tgm0000gn/T/ipykernel_33329/408742767.py:158: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Roc Auc,Pr Auc
1,0.245700,0.212520,0.949765,0.000000,0.665081,0.122183
2,0.235500,0.208407,0.949765,0.000000,0.723248,0.123348
3,0.232000,0.253487,0.948357,0.083333,0.703208,0.123079



[eval] validation metrics:


{'eval_loss': 0.25272297859191895, 'eval_accuracy': 0.9483568075117371, 'eval_f1': 0.08333333333333333, 'eval_roc_auc': 0.7042377148770449, 'eval_pr_auc': 0.13133164346734363, 'eval_runtime': 17.1799, 'eval_samples_per_second': 123.982, 'eval_steps_per_second': 7.8, 'epoch': 3.0}

[eval] test metrics & reports:
              precision    recall  f1-score   support

           0     0.9508    0.9972    0.9735      5059
           1     0.2632    0.0188    0.0351       266

    accuracy                         0.9484      5325
   macro avg     0.6070    0.5080    0.5043      5325
weighted avg     0.9165    0.9484    0.9266      5325

ROC AUC: 0.663471784818837
PR  AUC: 0.08983621532057373
Confusion matrix:
 [[5045   14]
 [ 261    5]]

Saved predictions to ./bert_text_model/test_predictions.csv
